In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score, r2_score
import numpy as np


In [5]:
# Define paths
base_dir = "D:/AI Algorithm/cucumber/"
train_dir = base_dir + "train/"
validate_dir = base_dir + "validation/"
test_dir = base_dir + "test/"

# Image Data Generators
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
validate_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load data
train_data = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='binary')
validate_data = validate_datagen.flow_from_directory(validate_dir, target_size=(224, 224), batch_size=32, class_mode='binary')
test_data = test_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32, class_mode='binary', shuffle=False)


Found 690 images belonging to 2 classes.
Found 230 images belonging to 2 classes.
Found 232 images belonging to 2 classes.


In [6]:
# Load pretrained VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification

# Create final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 15s 0us/step


In [8]:
# Train the model
history = model.fit(
    train_data,
    epochs=10,
    validation_data=validate_data
)


Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 123s 5s/step - accuracy: 0.6353 - loss: 0.6895 - val_accuracy: 0.7174 - val_loss: 0.4994
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 116s 5s/step - accuracy: 0.7488 - loss: 0.5198 - val_accuracy: 0.7565 - val_loss: 0.4343
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 116s 5s/step - accuracy: 0.7847 - loss: 0.4556 - val_accuracy: 0.8739 - val_loss: 0.3626
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 115s 5s/step - accuracy: 0.8649 - loss: 0.3763 - val_accuracy: 0.8696 - val_loss: 0.3253
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 116s 5s/step - accuracy: 0.8840 - loss: 0.3260 - val_accuracy: 0.8739 - val_loss: 0.2953
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 116s 5s/step - accuracy: 0.8838 - loss: 0.2993 - val_accuracy: 0.9261 - val_loss: 0.2299
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 115s 5s/step - accuracy: 0.9206 - loss: 0.2684 - val_accuracy: 0.8870 - val_loss: 0.2334
Epoch 8/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 114s 5s/step - accuracy: 0.9439 - loss: 0.2325 - val_accuracy: 0.9304 - v

In [9]:
# Predict on test data
y_pred_probs = model.predict(test_data)
y_pred = (y_pred_probs > 0.5).astype(int).ravel()
y_true = test_data.classes

# Classification Metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, zero_division=1)
recall = recall_score(y_true, y_pred, zero_division=1)
f1 = f1_score(y_true, y_pred, zero_division=1)
try:
    auc = roc_auc_score(y_true, y_pred_probs.ravel())
except ValueError:
    auc = 0.0
conf_matrix = confusion_matrix(y_true, y_pred)

# Regression Metrics
mae = mean_absolute_error(y_true, y_pred_probs.ravel())
mse = mean_squared_error(y_true, y_pred_probs.ravel())
rmse = np.sqrt(mse)
explained_var = explained_variance_score(y_true, y_pred_probs.ravel())
r2 = r2_score(y_true, y_pred_probs.ravel())
mbd = np.mean(y_pred_probs.ravel() - y_true)

# Print metrics
print("Classification Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"AUC: {auc:.4f}")
print(f"Confusion Matrix:\n{conf_matrix}\n")

print("Regression Metrics:")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Explained Variance Score: {explained_var:.4f}")
print(f"R^2 Score: {r2:.4f}")
print(f"Mean Bias Deviation (MBD): {mbd:.4f}")


8/8 ━━━━━━━━━━━━━━━━━━━━ 30s 3s/step
Classification Metrics:
Accuracy: 0.9612
Precision: 0.9645
Recall: 0.9819
F1-Score: 0.9731
AUC: 0.9898
Confusion Matrix:
[[ 60   6]
 [  3 163]]

Regression Metrics:
Mean Absolute Error (MAE): 0.1123
Mean Squared Error (MSE): 0.0384
Root Mean Squared Error (RMSE): 0.1959
Explained Variance Score: 0.8132
R^2 Score: 0.8114
Mean Bias Deviation (MBD): 0.0187


In [13]:
# Save the model
model.save('vgg_net16.h5')
# Save the model in the recommended Keras format
model.save("vgg_net16.keras")  # Save in the .keras format
print("Model saved successfully!")

Model saved successfully!
